# Implementation of SHA-1 Core on SoC Ported to Python

In [ ]:
# Internal constants definitions
ADDR_I_CS_DATA      = 0x10
BITS_I_CS_DATA      = 1
ADDR_I_WE_DATA      = 0x18
BITS_I_WE_DATA      = 1
ADDR_I_ADDRESS_DATA = 0x20
BITS_I_ADDRESS_DATA = 8
ADDR_I_WRITE_DATA   = 0x28
BITS_I_WRITE_DATA   = 32
ADDR_O_READ_DATA    = 0x30
BITS_O_READ_DATA    = 32
ADDR_O_READ_CTRL    = 0x34
ADDR_O_ERROR_DATA   = 0x38
BITS_O_ERROR_DATA   = 1
ADDR_O_ERROR_CTRL   = 0x3c

In [ ]:
# Load the overlay containing the IP
from pynq import Overlay

overlay = Overlay('/home/xilinx/sha1_overlay/sha1_overlay.bit')

In [ ]:
# See what attributes the overlay has
overlay?

In [ ]:
# Access the the sha1 attribute to create a driver for the IP
sha1_ip = overlay.sha1_control_0
sha1_ip?

In [ ]:
def check_name():
    # Get the name and version of the device
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name0 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name0)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 1)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name1 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name1)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 2)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    version = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(version)[2:]).decode())

In [ ]:
def read_word(address):
    '''
    Read a data word from the given address
    Return the data word that was obtained
    '''
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    while(sha1_ip.read(ADDR_O_READ_DATA) != 0):
        word = sha1_ip.read(ADDR_O_READ_DATA)
        break
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    return word

In [ ]:
def write_word(address, word):
    # Write the given word to the given address
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_WRITE_DATA, word)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 1)
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    sha1_ip.write(ADDR_I_WE_DATA, 0)

In [ ]:
def write_block(block):
    # Write the given block to the device
    mask = 0xffffffff000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
    write_word(0x10, (block & mask) >> 480);
    write_word(0x11, (block & (mask >> 32)) >> 448);
    write_word(0x12, (block & (mask >> 64)) >> 416);
    write_word(0x13, (block & (mask >> 96)) >> 384);
    write_word(0x14, (block & (mask >> 128)) >> 352);
    write_word(0x15, (block & (mask >> 160)) >> 320);
    write_word(0x16, (block & (mask >> 192)) >> 288);
    write_word(0x17, (block & (mask >> 224)) >> 256);
    write_word(0x18, (block & (mask >> 256)) >> 224);
    write_word(0x19, (block & (mask >> 288)) >> 192);
    write_word(0x1a, (block & (mask >> 320)) >> 160);
    write_word(0x1b, (block & (mask >> 352)) >> 128);
    write_word(0x1c, (block & (mask >> 384)) >> 96);
    write_word(0x1d, (block & (mask >> 416)) >> 64);
    write_word(0x1e, (block & (mask >> 448)) >> 32);
    write_word(0x1f, block & (mask >> 480));

In [ ]:
def read_digest():
    # Read the digest from the device
    # Return the digest data that was read
    digest_data_mask = 0xffffffff000000000000000000000000ffffffff
    mask = 0xffffffff00000000000000000000000000000000
    digest_data = (read_word(0x20) & digest_data_mask) << 128
    digest_data = digest_data | ((read_word(0x21) & digest_data_mask) << 96)
    digest_data = digest_data | ((read_word(0x22) & digest_data_mask) << 64)
    digest_data = digest_data | ((read_word(0x23) & digest_data_mask) << 32)
    digest_data = digest_data | read_word(0x24)
    return digest_data

In [ ]:
def create_block(msg):
    # Convert the given message into a 512-bit padded block to be processed
    # Return the block as an integer
    from bitstring import BitArray
    binMsg = ''.join(format(ord(ch), '08b') for ch in msg)
    binMsg = "0b" + binMsg
    arr = BitArray(binMsg)
    msgSize = len(arr)
    arr.append('0b1')
    padSize = (512 - 64) - len(arr)
    for i in range(padSize):
        arr.append('0b0')
    reserved = format(msgSize, '#066b')
    arr.append(reserved)
    return arr.int

In [ ]:
def test_sha1():
    # Check if the message digests from the software and hardware are the same
    import hashlib
    swDigest = int(hashlib.sha1(b"abc").hexdigest(), 16)
    print("Software SHA-1 Digest: " + str(swDigest))
    test_block = create_block("abc")
    write_block(test_block)
    write_word(0x08, 1)
    hwDigest = read_digest()
    print("Hardware SHA-1 Digest: " + str(hwDigest))
    
    if swDigest == hwDigest:
        print("Test Passed!")
    else:
        print("Test Failed!")

In [ ]:
# Run the test
test_sha1()